# PyCity Schools
* **Author:** Felipe Murillo
* **Date:** April 18, 2020

* **Description:** Analyze school data to make strategic decisions regarding future school budgets and priorities.

* **Required Input Files:** /Resources/schools_complete.csv & /Resources/students_complete.csv

## >> Observable Trends

1. Most students are doing better with reading than they are with math

2. Charter schools have an overwhelming better overall passing rate than district schools

3. Spending MORE money per student does NOT yield higher overall passing rates. Better overall grades happen when less money is thrown at the problem.

4. Reading and math scores do not significantly change across grades (they stay the same from 9th to 12th grade). May hint that if scores are to be improved, maybe changes need to happen in elementary and middle schools.

5. Large schools yield lower passing scores.

### *>>> Dependencies and Setup*

In [3]:
# Dependencies and Setup
import pandas as pd
import os

### *>>> Import Raw Data*

In [313]:
# File to Load (Remember to Change These)
school_data_to_load = os.path.join("../Resources/schools_complete.csv")
student_data_to_load = os.path.join("../Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


### *>>> District Summary*

In [249]:
# Calculate the total number of schools
n_Schools = school_data_complete["school_name"].value_counts().count()

# Calculate the total number of students
n_Students = school_data_complete["Student ID"].value_counts().count()

# Total budget
totBudget = school_data["budget"].sum()

# Average math score
avgMath = school_data_complete["math_score"].mean()

# Average reading score
avgRead = school_data_complete["reading_score"].mean()

# Students with passing math grades (count and percentage)
n_GoodMath_filter = school_data_complete["math_score"] >= 70
n_GoodMath = school_data_complete.loc[n_GoodMath_filter]["Student ID"].count()
p_GoodMath = n_GoodMath/n_Students * 100

# Students with passing reading grades
n_GoodRead_filter = school_data_complete["reading_score"] >= 70
n_GoodRead= school_data_complete.loc[n_GoodRead_filter]["Student ID"].count()
p_GoodRead = n_GoodRead/n_Students * 100

# Overall passing students
n_GoodOverall_filter = (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
n_GoodOverall = school_data_complete.loc[n_GoodOverall_filter]["Student ID"].count()
p_GoodOverall = n_GoodOverall/n_Students * 100

### *>>> <u>District Summary Table*</u>

In [250]:
# Compile summary table into a dataframe
district_summary_df = pd.DataFrame({
                                "Total Schools": n_Schools,
                                "Total Students": n_Students,
                                "Total Budget": totBudget,
                                "Average Math Score": avgMath,
                                "Average Reading Score": avgRead,
                                "% Passing Math": p_GoodMath,
                                "% Passing Reading": p_GoodRead,
                                "% Overall Passing": [p_GoodOverall]
                                })

# Format the summary table
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

# Display the formatted summary table
district_summary_df.style.hide_index().set_properties(**{'text-align': 'center'})

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
15,"39,170","$24,649,428.00",78.9854,81.8778,74.9809,85.8055,65.1723


### *>>> School Summary*

In [354]:
# Group Data by School
grpSchoolData = school_data_complete.groupby(by=["school_name"],as_index = True)

# Add number of students per school
tot_StudentsperSchool = grpSchoolData["Student ID"].count()

# Get school data (that can't be pulled from groupby data
typeSchool = school_data.set_index("school_name")["type"]
budgetSchool = school_data.set_index("school_name")["budget"]

# Per student budget
perStudentBudgetSchool = budgetSchool/tot_StudentsperSchool

# Average math and reading scores
avgMathSchools = (grpSchoolData["math_score"].sum())/tot_StudentsperSchool
avgReadSchools = (grpSchoolData["reading_score"].sum())/tot_StudentsperSchool

# Determine number of good math students
n_GoodMath_filter2 = school_data_complete["math_score"] >= 70
goodMath_perSchool = school_data_complete[n_GoodMath_filter2].groupby(by="school_name")
p_goodMathSchool = goodMath_perSchool["Student ID"].count()/tot_StudentsperSchool *100

# Determine number of good reading students
n_GoodRead_filter2 = school_data_complete["reading_score"] >= 70
goodRead_perSchool = school_data_complete[n_GoodRead_filter2].groupby(by="school_name")
p_goodReadSchool = goodRead_perSchool["Student ID"].count()/tot_StudentsperSchool *100

# Determine number of good overall students
n_GoodOverall_filter2 = (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
goodOverall_perSchool = school_data_complete[n_GoodOverall_filter2].groupby(by="school_name")
p_goodOverallSchool = goodOverall_perSchool["Student ID"].count()/tot_StudentsperSchool *100


### *>>> <u>School Summary Table</u>*

In [421]:
# Compile school summary table into a dataframe
schools_summary_df = pd.DataFrame({       
                                "School Type": typeSchool,
                                "Total Students": tot_StudentsperSchool,
                                "Total Budget": budgetSchool,
                                "Per Student Budget": perStudentBudgetSchool,
                                "Average Math Score": avgMathSchools,
                                "Average Reading Score": avgReadSchools,
                                "% Passing Math": p_goodMathSchool,
                                "% Passing Reading": p_goodReadSchool,
                                "% Overall Passing": p_goodOverallSchool
                                })

# Format the summary table
schools_summary_df["Total Students"] = schools_summary_df["Total Students"].map("{:,}".format)
schools_summary_df["Total Budget"] = schools_summary_df["Total Budget"].map("${:,.2f}".format)
schools_summary_df["Per Student Budget"] = schools_summary_df["Per Student Budget"].map("${:,.2f}".format)

# Display the formatted summary table
schools_summary_df.style.set_properties(**{'text-align': 'center'})

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.0484,81.034,66.6801,81.9333,54.6423
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.0619,83.9758,94.1335,97.0398,91.3348
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7118,81.158,65.9885,80.7392,53.2045
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1026,80.7463,68.3096,79.299,54.2899
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.3515,83.8168,93.3924,97.139,90.5995
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.2898,80.9344,66.753,80.863,53.5275
Holden High School,Charter,427,"$248,087.00",$581.00,83.8033,83.815,92.5059,96.2529,89.2272
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6294,81.1827,65.6839,81.3164,53.5139
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.0725,80.9664,66.0576,81.2224,53.5392
Pena High School,Charter,962,"$585,858.00",$609.00,83.8399,84.0447,94.5946,95.9459,90.5405


### >>> Top Performing Schools (By % Overall Passing)

In [303]:
# Format the summary table
overall_top_df = schools_summary_df.sort_values("% Overall Passing",ascending = False)

# Display the formatted summary table
overall_top_df.style.set_properties(**{'text-align': 'center'})
overall_top_df.head()

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


### >>> Bottom Performing Schools (By % Overall Passing)

In [304]:
# Format the summary table
overall_bottom_df = schools_summary_df.sort_values("% Overall Passing",ascending = True)

# Display the formatted summary table
overall_bottom_df.style.set_properties(**{'text-align': 'center'})
overall_bottom_df.head()

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


### >>> Math Scores by Grade

In [329]:
# Series for 9th Grade
ninth_filter = student_data["grade"]== "9th"
ninth_grade = student_data[ninth_filter].set_index("school_name")
ninth_grade_math = ninth_grade["math_score"].groupby("school_name").mean()

# Series for 10th Grade
tenth_filter = student_data["grade"]== "10th"
tenth_grade = student_data[tenth_filter].set_index("school_name")
tenth_grade_math = tenth_grade["math_score"].groupby("school_name").mean()

# Series for 11th Grade
eleventh_filter = student_data["grade"]== "11th"
eleventh_grade = student_data[eleventh_filter].set_index("school_name")
eleventh_grade_math = eleventh_grade["math_score"].groupby("school_name").mean()

# Series for 12th Grade
twelfth_filter = student_data["grade"]== "12th"
twelfth_grade = student_data[twelfth_filter].set_index("school_name")
twelfth_grade_math = twelfth_grade["math_score"].groupby("school_name").mean()

In [334]:
# Compile math summary table
mathByGrades = pd.DataFrame({
                            "9th": ninth_grade_math,
                            "10th":tenth_grade_math,
                            "11th":eleventh_grade_math,
                            "12th":twelfth_grade_math
                            })

# Display the formatted summary table
mathByGrades.style.set_properties(**{'text-align': 'center'})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.0837,76.9968,77.5156,76.4922
Cabrera High School,83.0947,83.1545,82.7656,83.2775
Figueroa High School,76.403,76.54,76.8843,77.1514
Ford High School,77.3613,77.6723,76.9181,76.18
Griffin High School,82.044,84.2291,83.8421,83.3562
Hernandez High School,77.4385,77.3374,77.136,77.1866
Holden High School,83.7874,83.4298,85,82.8554
Huang High School,77.0273,75.9087,76.4466,77.2256
Johnson High School,77.1879,76.6911,77.4917,76.8632


### >>> Reading Scores by Grade 

In [332]:
# Series for 9th Grade
ninth_grade_read = ninth_grade["reading_score"].groupby("school_name").mean()

# Series for 10th Grade
tenth_grade_read = tenth_grade["reading_score"].groupby("school_name").mean()

# Series for 11th Grade
eleventh_grade_read = eleventh_grade["reading_score"].groupby("school_name").mean()

# Series for 12th Grade
twelfth_grade_read = twelfth_grade["reading_score"].groupby("school_name").mean()

In [333]:
# Compile reading summary table
readingByGrades = pd.DataFrame({
                            "9th": ninth_grade_read,
                            "10th":tenth_grade_read,
                            "11th":eleventh_grade_read,
                            "12th":twelfth_grade_read
                            })

# Display the formatted summary table
readingByGrades.style.set_properties(**{'text-align': 'center'})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.3032,80.9072,80.9456,80.9125
Cabrera High School,83.6761,84.2532,83.7884,84.288
Figueroa High School,81.1986,81.4089,80.6403,81.3849
Ford High School,80.6327,81.2627,80.4036,80.6623
Griffin High School,83.3692,83.7069,84.2881,84.0137
Hernandez High School,80.8669,80.6601,81.3961,80.8571
Holden High School,83.6772,83.3246,83.8155,84.6988
Huang High School,81.2903,81.5124,81.4175,80.306
Johnson High School,81.2607,80.7734,80.616,81.2276


### >>> Scores by School Spending

In [386]:
# Create an spending range bin
bins = [0,584,629,644,675]

# Create spending range labels
grp_labels = ["<$584","$585-629","$630-644","$645-675"]

# Pull spending summary calcuated earlier
spending_summary_df = pd.DataFrame({       
                                "Per Student Budget": perStudentBudgetSchool,
                                "Average Math Score": avgMathSchools,
                                "Average Reading Score": avgReadSchools,
                                "% Passing Math": p_goodMathSchool,
                                "% Passing Reading": p_goodReadSchool,
                                "% Overall Passing": p_goodOverallSchool
                                })

# Cut spending summary into desired bins and make bins the indices
spending_summary_df["Spending ranges (per student)"] = pd.cut(spending_summary_df["Per Student Budget"],bins,labels=grp_labels)
spending_summary_df.set_index("Spending ranges (per student)",inplace = True)

# Group by bins and remove per student budget column
grpSpending = spending_summary_df.drop(columns =["Per Student Budget"]).groupby("Spending ranges (per student)")

# Determine avgs
avgMathSpending = grpSpending["Average Math Score"].mean()
avgReadSpending = grpSpending["Average Reading Score"].mean()
avgMathPass = grpSpending["% Passing Math"].mean()
avgReadPass = grpSpending["% Passing Reading"].mean()
overallPass = grpSpending["% Overall Passing"].mean()

In [419]:
# Compile summary table
scores_by_spending = pd.DataFrame({       
                                "Average Math Score": avgMathSpending,
                                "Average Reading Score": avgReadSpending,
                                "% Passing Math": avgMathPass,
                                "% Passing Reading": avgReadPass,
                                "% Overall Passing": overallPass
                                })

scores_by_spending.style.set_properties(**{'text-align': 'center'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending ranges (per student),,,,,
<$584,83.4554,83.9338,93.4601,96.6109,90.3695
$585-629,81.8998,83.1553,87.1335,92.7182,81.4186
$630-644,78.5189,81.6245,73.4842,84.3918,62.8577
$645-675,76.9972,81.0278,66.1648,81.134,53.5269


### >>> Scores by School Size

In [426]:
# Create an school size range bin
bins = [0,1000,2000,5000]

# Create school size range labels
grp_labels = ["Small (<1000)","Medium (1000 - 2000)","Large (2000-5000)"]

# Pull school size summary calculated earlier
size_summary_df = pd.DataFrame({       
                                "Total Students": tot_StudentsperSchool,
                                "Average Math Score": avgMathSchools,
                                "Average Reading Score": avgReadSchools,
                                "% Passing Math": p_goodMathSchool,
                                "% Passing Reading": p_goodReadSchool,
                                "% Overall Passing": p_goodOverallSchool
                                })

# Cut size summary into desired bins and make bins the indices
size_summary_df["School Size"] = pd.cut(size_summary_df["Total Students"],bins,labels=grp_labels)
size_summary_df.set_index("School Size",inplace = True)

# Group by bins and remove per student budget column
grpSize = size_summary_df.drop(columns =["Total Students"]).groupby("School Size")

# Determine avgs
avgMathSize = grpSize["Average Math Score"].mean()
avgReadSize = grpSize["Average Reading Score"].mean()
avgMathPassSize = grpSize["% Passing Math"].mean()
avgReadPassSize = grpSize["% Passing Reading"].mean()
overallPassSize = grpSize["% Overall Passing"].mean()

In [431]:
# Compile summary table
scores_by_size = pd.DataFrame({       
                                "Average Math Score": avgMathSize,
                                "Average Reading Score": avgReadSize,
                                "% Passing Math": avgMathPassSize,
                                "% Passing Reading": avgReadPassSize,
                                "% Overall Passing": overallPassSize
                                })

scores_by_size.style.set_properties(**{'text-align': 'center'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.8216,83.9298,93.5502,96.0994,89.8839
Medium (1000 - 2000),83.3747,83.8644,93.5997,96.7907,90.6215
Large (2000-5000),77.7464,81.3445,69.9634,82.7666,58.286


### >>> Scores by School Type

* Perform the same operations as above, based on school type

In [437]:
# Pull school size summary calculated earlier
type_summary_df = pd.DataFrame({       
                                "School Type": typeSchool,
                                "Average Math Score": avgMathSchools,
                                "Average Reading Score": avgReadSchools,
                                "% Passing Math": p_goodMathSchool,
                                "% Passing Reading": p_goodReadSchool,
                                "% Overall Passing": p_goodOverallSchool
                                })

# Group by bins and remove per student budget column
grpType = type_summary_df.groupby("School Type")

# Determine avgs
avgMathType = grpType["Average Math Score"].mean()
avgReadType = grpType["Average Reading Score"].mean()
avgMathPassType = grpType["% Passing Math"].mean()
avgReadPassType = grpType["% Passing Reading"].mean()
overallPassType = grpType["% Overall Passing"].mean()

In [438]:
# Compile summary table
scores_by_type = pd.DataFrame({       
                                "Average Math Score": avgMathType,
                                "Average Reading Score": avgReadType,
                                "% Passing Math": avgMathPassType,
                                "% Passing Reading": avgReadPassType,
                                "% Overall Passing": overallPassType
                                })

scores_by_type.style.set_properties(**{'text-align': 'center'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.4739,83.8964,93.6208,96.5865,90.4322
District,76.9567,80.9666,66.5485,80.7991,53.6722
